In [1]:
import scipy.io as scio
from torch_geometric.data import Data
import torch
import os
from tqdm import tqdm

In [2]:
data_list = torch.load('pretrain_desco.pt')

In [ ]:
# use EVOKE to compute ground truth
# 2.1 generate edge_list files EVOKE needs 
edgelist_graph_path = 'evoke/wrappers/edgelist_graph'
if not os.path.exists(edgelist_graph_path):
    os.mkdir(edgelist_graph_path)
for i, g in tqdm(enumerate(data_list), total=len(data_list), desc='Generating edge list files'):
    with open(f'{edgelist_graph_path}/rw_{i}.edgelist', 'w') as f:
        f.write(f'{g.num_nodes} {g.num_edges//2}\n')
        for edge in g.edge_index.T:
            if edge[0] < edge[1]:
                f.write(f'{edge[0]} {edge[1]}\n')
print('Edge list files generated')

In [3]:
# 2.2 compute the ground truth using 'run_ego.py' with EVOKE
output_evoke_path = 'evoke/wrappers/output_evoke'
output_graph_path = 'evoke/wrappers/output_graph'
# evoke_wrappers_path = 'evoke/wrappers/'
# run_evoke_name = 'run_evoke.py'
# if not os.path.exists(output_evoke_path):
#     os.mkdir(output_evoke_path)
# if os.path.exists(output_graph_path):
#     os.mkdir(output_graph_path)
# # run the EVOKE to compute the ground truth
# os.system(f'cd {evoke_wrappers_path} && python {run_evoke_name} graph')
# read in the ground truth and add gt_induced_le5 & gt_noninduced_le5 to the graphs
for i, g in tqdm(enumerate(data_list), desc='Adding ground truth to graphs'):
    induced = torch.load(f'{output_graph_path}/induced_rw_{i}.pt')
    g.gt_induced_le5 = induced

Adding ground truth to graphs: 1827it [00:01, 1737.42it/s]


In [4]:
dataset = {
    'train': data_list,
    'val': data_list,
    'test': data_list
}
torch.save(dataset, 'pretrain_desco.pt')

In [11]:
for g in data_list:
    g.y = g.gt_induced_le5[:, 1:]
    del g.gt_induced_le5
    g.x = torch.ones(g.num_nodes, 1)

In [13]:
print(data_list[1].x, data_list[1].y)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]) tensor([[25, 20,  0, 25,  0, 43, 39, 13,  0,  0, 11,  0,  0, 31,  0,  0, 52,  5,
          0,  0,  0, 17, 10, 21,  0, 43,  4, 13,  3],
        [16,  3,  5, 23,  0, 26, 11,  1,  6,  5, 12,  9,  1, 29,  0,  1, 37,  6,
          0,  0,  1, 10,  4,  8,  0, 14,  0,  2,  0],
        [15,  1,  2, 27,  0, 23,  7,  0,  0,  4, 12,  6,  0, 42,  0,  1, 36,  0,
          0,  0,  0, 11,  6,  2,  0, 10,  0,  0,  0],
        [16,  8,  9, 12,  2, 28, 18,  5,  7,  2,  5,  7,  3, 19,  1,  2, 36,  5,
          3,  0,  3,  5,  9, 14,  1, 16,  4,  8,  1],
        [16,  2, 14, 14,  2, 25,  9,  0,  5,  8,  8, 18,  2, 14,  1,  4, 34,  1,
          4,  0,  6,  3,  7, 14,  1,  9,  2,  0,  0],
        [17, 11,  6, 11,  3, 29, 22,  8,  3,  1,  5, 10,  1, 13,  0,  3, 41,  1,
          3,  0,  4,  5, 10, 10,  2, 24,  5, 10,  2],
        [21, 18,  6, 14,  1, 39, 32

In [16]:
data_dict = [g.to_dict() for g in data_list]

In [17]:
dataset_compatible = {
    'train': data_dict,
    'val': data_dict,
    'test': data_dict
}

In [1]:
print(dataset_compatible['train'][2])

NameError: name 'dataset_compatible' is not defined

In [21]:
torch.save(dataset_compatible, 'dataset_compatible.pt')

In [4]:
dataset_compatible = torch.load('dataset_compatible.pt')

In [6]:
print(dataset_compatible['train'][1])

{'x': tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]), 'edge_index': tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  2,  2,  3,  3,  3,
          3,  3,  4,  4,  4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,
          6,  6,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,
          9, 10, 10, 10, 10, 10],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  6,  7,  0,  6,  0,  6,  8,
          9, 10,  0,  7,  8,  0,  6,  7,  8,  9, 10,  0,  1,  2,  3,  5,  7,  8,
          9, 10,  0,  1,  4,  5,  6,  9,  0,  3,  4,  5,  6, 10,  0,  3,  5,  6,
          7,  0,  3,  5,  6,  8]]), 'y': tensor([[25, 20,  0, 25,  0, 43, 39, 13,  0,  0, 11,  0,  0, 31,  0,  0, 52,  5,
          0,  0,  0, 17, 10, 21,  0, 43,  4, 13,  3],
        [16,  3,  5, 23,  0, 26, 11,  1,  6,  5, 12,  9,  1, 29,  0,  1, 37,  6,
          0,  0,  1, 10,  4,  8,  0, 14,  0,  2